# Descrizione del problema
Il problema riguarda lo **scheduling**. Sono dati $G$ ingegneri che lavorono per $L$ giorni. La realtà è composta da 3 elementi:
* **Features**: sono le funzionalità implementate nei prodotti;
* **Services**: sono i programmi che sono in esecuzione sui server;
* **Binaries**: modellano un server. Ogni *binary* può avere in esecuzione più servizi;

## Relazioni tra le entità
Una **Feature** è contraddistinta dal numero di utenti che riesce a soddisfare e da un insieme di servizi in cui deve essere implementata. Una **Feature** è disponibile ai clienti quando viene implementata in ogni servizio da cui dipende.
Un **Service** è presente su **uno e un solo binary**, ma su ogni **binary** è possibile inserire più **Services**.

### Lettura input da file
Come prima cosa vengono letti i dati input e si effettua il parsing della prima riga per avere la dimensione del problema.

In [68]:
fname = "data/e.txt"

# lettura dei dati dal file di input
with open(fname, "r") as f:
    lines = f.readlines()
    
# parsing della prima riga: dimensione del problema
L, G, S, B, F, N = [int(k) for k in lines[0].strip().split(" ")]


In [69]:
# modellazione del sistema
from dataclasses import dataclass
from typing import List, Tuple, Dict, Set

score: int = 0
daily_schedule: List[Tuple[int, int]] = [] # giorno, binary

@dataclass
class Binary:
    id: int
    services: List[str]
    working_on: int
    locked: bool
binaries: List[Binary] =  [None] * B
service_to_binaries: Dict[str, int] = {}

@dataclass
class Service:
    name: str
    binary: int
    impl_features: List[str]

services: List[Service] = []

@dataclass
class Feature:
    idx: int
    name: str
    difficulty: int
    users: int
    dep_services: List[List]
    completed: bool
    binaries: Set[int]
    def populate_binaries(self):
        for (service, done) in self.dep_services:
            b = service_to_binaries[service]
            self.binaries.add(b)


features: List[Feature] = []
completed_features: List[Feature] = []

@dataclass
class Engineer:
    id: int
    avail: int
    ops: List[Tuple[str, List]]

    def add_op(self, op_name: str, day: str, duration: int, values: List[int]):
        if day > self.avail:
            self.ops.append(("wait", [day - self.avail]))

        affected_binaries = []
        if op_name == "move":
            service = services[values[0]]
            b1 = service_to_binaries[service.name]
            affected_binaries.append(b1, values[1])
        elif op_name == "impl":
            affected_binaries.append(values[1])
        elif op_name not in ["new", "wait"]:
            raise Exception(f"operation {op_name} not supported")

        # controllo se un binary coinvolto nell'operazione è bloccato
        # se si aggiungo una wait pari al massimo di quelli schedulati
        locked = [b for b in affected_binaries if binaries[b].locked]
        skip = []
        for l in locked:
            for ds in daily_schedule:
                if daily_schedule[ds] == l:
                    skip.append(ds)
        else:
            if len(skip) > 0:
                self.ops.append("wait", [day + skip])

        self.avail += day + duration
        self.ops.append((op_name, values))
engineers: List[Engineer] = []

### Parsing dell'input
In questa sezione, viene effettuato il parsing dei dati in ingresso creando:
* Lista dei binary: tutti i binary del sistema; per ogni binary si tiene traccia della list dei service installati e del numero di ingegneri che lavora al momento;
* Lista dei service; tutti i service del sistema. Per ogni service si tiene conto delle feature implementate al momento;
* Lista delle feature: tutte le feature del sistema. Ogni feature contiene una lista coppie (stringhe, booleani) in cui si tiene conto dei servizi da implementare e del loro stato (implementato si/no);

In [70]:
for i in range(G):
    engineers.append(Engineer(i, 0, []))

for (service, binary) in [line.strip().split() for line in lines[1:S+1]]:
    bid = int(binary)
    b = binaries[bid]
    if b is None:
        b = Binary(bid, [service], 0, False)
        binaries[bid] = b
    else:
        b.services.append(service)

    s = Service(service, bid, [])
    services.append(s)
    service_to_binaries[service] = bid
idx = 0
for i, feature in enumerate(lines[1+S:]):
    # leggo solo le righe pari
    if i % 2 ==0:
        feature, num_services, difficulty, users = feature.strip().split(" ")
        s = [[s, False] for s in lines[1+S+1+i].split()]
        f = Feature(idx, feature, int(difficulty), int(users), s, False, set())
        f.populate_binaries()
        features.append(f)
        idx += 1

print(engineers, binaries, service_to_binaries, features, services, sep="\n")

[Engineer(id=0, avail=0, ops=[]), Engineer(id=1, avail=0, ops=[]), Engineer(id=2, avail=0, ops=[]), Engineer(id=3, avail=0, ops=[]), Engineer(id=4, avail=0, ops=[]), Engineer(id=5, avail=0, ops=[]), Engineer(id=6, avail=0, ops=[]), Engineer(id=7, avail=0, ops=[]), Engineer(id=8, avail=0, ops=[]), Engineer(id=9, avail=0, ops=[])]
[Binary(id=0, services=['sf', 'sbc', 'sda', 'sdb', 'sdh', 'sdj', 'see', 'sef', 'sfi', 'sfj', 'shd', 'shj', 'sif', 'sje'], working_on=0, locked=False), Binary(id=1, services=['sbj', 'sei', 'sfb', 'sge', 'shb', 'sij', 'sjc'], working_on=0, locked=False), Binary(id=2, services=['sbi', 'sgb', 'shg', 'sie', 'sjd'], working_on=0, locked=False), Binary(id=3, services=['se', 'sh', 'sbb', 'scf', 'sdd', 'sde', 'sdf', 'sdi', 'seg', 'sff', 'sgh', 'she', 'sib', 'sji'], working_on=0, locked=False), Binary(id=4, services=['sbd', 'sgc', 'sjj'], working_on=0, locked=False), Binary(id=5, services=['sj', 'sba', 'sbe', 'sbf', 'sca', 'scg', 'sea', 'sfc', 'sfg', 'sgf', 'sih', 'sii']

## Attività
Di seguito sono illustrate le mosse che un ingegnere può effettuare.

Ciascuna mossa è caratterizzata da una durata. Per ogni mossa è definita una procedura per eseguirla sui dati di input. Ogni procedura ha la seguente firma:
* input:
    * ingegnere su cui schedulare l'attività;
    * giorno in cui effettuare l'attività;
    * feature, binary, service su cui effettuare l'attività;
* output:
    * durata dell'attività;

### Implementazione di una nuova feature
Su un *binary* $B_j$ può un ingegnere implementare tutti i servizi per una feature $F_i$. Questa mossa ha una durata $D_{Fi} + R_{Bj} + C_{Bj}$. Dove $D_{Fi}$ è data dalla complessità della **Feature**, $R_{Bj}$ è dato dal numero di servizi totali presenti nel *binary*, $C_{Bj}$ ingegneri che stanno già lavorando su $B_j$ nel giorno in cui viene *schedulata* l'attività. Inoltre, ogni ingegnere può implementare la stessa **Feature** su **binary** diversi. Infine, se più ingegneri sono *schedulati* per lavorare sullo stesso **binary** il primo della soluzione viene schedulato e conta per gli altri nel fattore $C_{Bj}$;


In [71]:
def implement_feature(e: Engineer, day: int, f: int, b: int) -> int:
    assert(b >= 0 and b < B)
    assert(day >=0 and day < L)
    assert(f >= 0 and f < F)
    global score
    feature = features[f]
    
    duration  = feature.difficulty + len(binaries[b].services) + binaries[b].working_on
    for service in feature.dep_services:
        s_name = service[0]
        done = service[1]
        if service_to_binaries[s_name] == b and not done:
            service[1] = True
    
    for (service, done) in feature.dep_services:
        if not done:
            break
    else:
        feature.completed = True
        features.remove(feature)
        completed_features.append(feature)
        # print("DEBUG: completed", feature, "multiply", 1 if  L-(day + duration) >= 0 else 0  )
        # print("DEBUG: remaining", len(features))
        # print("DEBUG: completed", completed_features)
        score += feature.users * max(0, L-(day + duration)) 
    e.add_op("impl", day, duration, [f, b])
    daily_schedule.append((day + duration, b))
    binaries[b].working_on+=1
    
    return duration

### Migrazione di un service
un ingegnere può spostare un servizio $S_i$ da un *binary* $B_j$ a $B_k$, mantenendo le funzionalità implementate. Per fare questa mossa, un ingegnere impiega $\max (R_{bj}, R_{bk})$ giorni dove $R_{bj}, R_{bk}$ sono i servizi in esecuzione rispettivamente su $B_j$ a $B_k$. Quest'azione però è bloccante: nessun ingegnere può lavorare su $B_j$ e $B_k$ quando è in corso una migrazione; 

In [72]:
def move_service(e: Engineer, day: int, s: int, dst: int) -> int:
    assert(service_to_binaries[s] != dst)
    assert(day >= 0 and  day < L)
    assert(dst >= 0 and dst < B)
    service = services[s]
    src = service.binary
    duration  = max(len(binaries[src].services), len(binaries[dst].services))
    service_to_binaries[service.name] = dst
    binaries[src].services.remove(s)
    binaries[dst].services.append(s)
    binaries[src].locked = True
    binaries[dst].locked = True
    e.add_op("move", day, duration, [service.name, src, dst])
    return duration

### Nuovo binary
Un ingegnere può creare un nuovo binary senza nessun servizio al suo interno. Il costo per creare è un nuovo binary è fisso e vale $N$.


In [73]:
def new_binary(e: Engineer, day: int)-> int:
    assert(day >=0 and day < L)
    binaries.append(Binary(len(binaries)))
    e.add_op("new", day, N, [])
    return N

### Idle
Un ingegnere può essere in idle per un numero di giorni $1\leq W \leq L$

In [74]:
def wait(e: Engineer, day: int, duration: int) -> int:
    assert(day >=0 and day < L)
    e.add_op("wait", day, duration, [])
    return 1

## Euristica 
In una fase preliminare, l'algoritmo ordina le feature per numero di utenti soddisfatti. In questo approccio, si assegna un punteggio maggiore ai servizi che risiedono sulla stessa macchina.

In [75]:

m = max([x.users for x in features])
features.sort(key=lambda x: x.users-m*len(x.binaries), reverse=True)
# print(features[:10])

for day in range(L):
    while len(daily_schedule) > 0 and daily_schedule[-1][0] <= day:
        # migrazione
        if len(daily_schedule[-1]) == 3:
            b1 = daily_schedule[-1][1]
            b2 = daily_schedule[-1][2]
            binaries[b1].locked = False 
            binaries[b2].locked = False 
        elif len(daily_schedule[-1]) == 2:
            bid: int = daily_schedule[-1][1] 
            binaries[bid].working_on -=1
        else:
            raise Exception("unknown schedule event", daily_schedule[-1])
        daily_schedule.pop()
    
    for engineer in engineers:
        if engineer.avail <= day:
            if len(features) == 0:
                break

            # b = service_to_binaries[features[-1].dep_services[0][0]]
            # b = random.randint(0, len(binaries) - 1)
            # f = random.randint(0, len(features) - 1)
            # print("impl", f.name, f.idx, b)
            # implement_feature(engineer, day, len(features) - 1, b)
            # if features[-1].completed:
                #continue
            f: Feature = features[0]
            for bin in f.binaries:
                # next_b = service_to_binaries[dep[0]]
                # if next_b != b and not dep[1]:
                    implement_feature(engineer, day, 0, bin)
                    if f.completed:
                        break
           

all_features = features + completed_features
num_eng: int = len([engineer for engineer in engineers if len(engineer.ops) > 0])
print(num_eng)
for engineer in engineers:
    if len(engineer.ops) == 0:
        continue
    print(engineer.id)
    for op in engineer.ops:
        print(op[0], end=" ")
        if op[0] == "impl":
            f = all_features[op[1][0]]
            print(f.name, op[1][1])
        elif op[0] == "move":
            print(op[1][0], op[1][2])
        elif op[0] == "wait":
            print(op[0], op[1][0])

print(score)
# print("features", features)
# print("impl features:", impl_features)
# sus = [f.users for f in impl_features]
# print(sum(sus))

10
0
impl sgjcigfc 5
impl sgjcigfc 3
impl sgjcigfc 3
impl sgjcigfc 8
1
impl sgjcigfc 3
impl sgjcigfc 3
impl sgjcigfc 3
impl sgjcigfc 7
2
impl sgjcigfc 0
impl sgjcigfc 7
impl sgjcigfc 7
impl sgjcigfc 3
3
impl sgjcigfc 0
impl sgjcigfc 9
impl sgjcigfc 9
impl sgjcigfc 6
4
impl sgjcigfc 2
impl sgjcigfc 9
impl sgjcigfc 8
impl sgjcigfc 1
5
impl sgjcigfc 0
impl sgjcigfc 6
impl sgjcigfc 3
impl sgjcigfc 1
6
impl sgjcigfc 0
impl sgjcigfc 8
impl sgjcigfc 8
impl sgjcigfc 0
7
impl sgjcigfc 3
impl sgjcigfc 7
impl sgjcigfc 5
impl sgjcigfc 8
8
impl sgjcigfc 0
impl sgjcigfc 0
impl sgjcigfc 3
impl sgjcigfc 3
9
impl sgjcigfc 5
impl sgjcigfc 2
impl sgjcigfc 9
impl sgjcigfc 5
17179215
